In [1]:
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [2]:
with open('data/jfk.pkl', 'rb') as f:
    jfk = pkl.load(f)

In [3]:
with open('data/ctl_pk.pkl', 'rb') as f:
    park = pkl.load(f)

In [45]:
with open('data/turnstile.pkl', 'rb') as f:
    turnstile = pkl.load(f)

In [5]:
with open('data/ac_delays.pkl','rb') as f:
    delays = pkl.load(f)

In [6]:
delays

,datetime,subject,message
48,2021-09-30 15:12:00,"Update: A and Rockaway Park S Trains, Delays",Expect longer waits for A /Rockaway Rockaway P...
50,2021-09-30 14:57:00,"BKLYN, MANH, C Train, Delays",You may wait longer for a Euclid Av-bound C tr...
65,2021-09-30 11:01:00,"Update: MANH, B and C Trains, Delays",Southbound B and C trains are proceeding after...
68,2021-09-30 10:50:00,"MANH, B and C Trains, Delays",Southbound B and C trains are delayed while we...
72,2021-09-30 09:05:00,"Update: MANH, B and C Trains, Delays",Northbound B/C trains are proceeding with dela...
...,...,...,...
215922,2010-01-04 18:38:00,"BKLYN, A & C trains, smoke condition",QNS-bound A & Euclid-bound C trains run on F l...
215927,2010-01-04 12:54:00,"UPDATED: MANH, A Train, Police Investigation",A train service has resumed with residual dela...
215928,2010-01-04 12:14:00,"MANH, A Train, Police Investigation",Ozone Park-bound A trains are running local fr...
215950,2010-01-01 09:17:00,"UPDATED: MANH, C train, Mechanical Problems",C train service has resumed with residual delays.


In [40]:
turnstile

0    2014-10-01
1    2014-11-01
2    2014-12-01
3    2015-01-01
4    2015-02-01
        ...    
80   2021-06-01
81   2021-07-01
82   2021-08-01
83   2021-09-01
84   2021-10-01
Name: date, Length: 85, dtype: datetime64[ns]

In [41]:
first_col = turnstile.pop('date')

In [42]:
turnstile.insert(0, 'date', first_col)

In [46]:
turnstile

,date,103_st_entr,103_st_exits,104_st_entr,104_st_exits,111_st_entr,111_st_exits,116_st_entr,116_st_exits,125_st_entr,...,spring_st_entr,spring_st_exits,utica_av_entr,utica_av_exits,van_siclen_av_entr,van_siclen_av_exits,w_4_st_wash_sq_entr,w_4_st_wash_sq_exits,world_trade_ctr_entr,world_trade_ctr_exits
0,2014-10-01,1339771034,1175549287,312180061,439393083,119036167,159289252,630340462,715496936,34600927541,...,338734016,91151565,379610520,352333513,536869121,289711145,265543250,650228597,417383539,155881610
1,2014-11-01,1938985908,1699319389,341887665,480915802,1542005,1933245,100165647,104067709,2304342370,...,500276640,134904923,427692220,396880757,623155442,337136512,380111665,931137920,587099717,219494659
2,2014-12-01,2015105131,1763593480,413351610,581215713,1594740,1951644,107104106,110268079,2340107113,...,530404721,143454032,595325444,553710871,836184568,451403418,235641584,526378086,607005384,227169486
3,2015-01-01,2047822007,1789884464,479972302,674538985,1577794,1999311,130839543,136310022,4672301998,...,559846009,152468393,610177079,567261067,844790583,454299234,24350890,65934335,637029840,238563612
4,2015-02-01,1839556877,1605513283,415081380,582665790,3552727,2663839,129556252,135031837,4206643577,...,492295275,132958926,549008461,509519078,768828853,412998135,22324713,64198566,564490383,211646611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2021-06-01,2728777474,2165257658,86834687,53216060,9059741280,3019898880,13343992,12451586,56461749,...,1219301529,312525502,1199644962,1173194301,1257504992,648242714,61004393,189249313,176436442,325257672
81,2021-07-01,2808432153,2229252113,807771,710081,9263623316,3089554922,14818207,14259441,61089886,...,1249569515,320617894,1253478596,1225081537,1301960949,671656680,43119749,134905643,183450148,340369820
82,2021-08-01,2813225284,2233995712,2349746,1984528,9419753742,3144993369,16333822,16442483,64821086,...,1253043179,321901544,1242989450,1218495684,1297765490,669929580,3844317,14464713,188579058,352413376
83,2021-09-01,2712397275,2154883947,3702371,3082717,9160406992,3053453312,16525011,16700510,64401219,...,1222568070,314595666,1209023751,1188458028,1258435209,650203186,8130813,31570270,183590669,344311417


In [8]:
jfk

,min_temp,avg_temp,max_temp,major_conditions,minor_conditions,neutral_conditions,normal/unknown_conditions,precipitation,avg_wind_spd,fastest_wind_time,peak_gust_time,snowfall_(in),snow_depth_(in),fastest_5_sec_wind,direction_fastest_wind
date,,,,,,,,,,,,,,,
2010-01-01,32,NaN,41,1,0,0,0,0.04,5.37,11:59 PM,11:55 PM,0.0,0.0,23.0,nw
2010-01-02,17,NaN,33,1,0,0,0,0.00,23.71,11:30 PM,8:39 PM,0.1,0.0,45.0,nw
2010-01-03,18,NaN,24,1,0,0,0,0.00,25.50,11:18 AM,12:14 PM,0.0,0.0,47.0,nw
2010-01-04,20,NaN,32,0,0,0,1,0.00,17.67,4:49 AM,4:12 AM,0.0,0.0,31.1,nw
2010-01-05,21,NaN,32,0,0,0,1,0.00,15.88,3:58 PM,3:57 PM,0.0,0.0,30.0,nw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,61,69.0,80,0,0,0,1,0.00,14.09,NaN,NaN,0.0,0.0,27.1,s
2021-09-28,62,69.0,76,0,0,1,0,0.09,10.96,NaN,NaN,0.0,0.0,23.9,w
2021-09-29,54,64.0,72,0,0,0,1,0.00,10.51,NaN,NaN,0.0,0.0,25.9,n


In [9]:
park

,min_temp,max_temp,major_conditions,minor_conditions,neutral_conditions,normal/unknown_conditions,precipitation,avg_wind_spd,fastest_wind_time,peak_gust_time,snowfall_(in),snow_depth_(in),fastest_5_sec_wind,direction_fastest_wind
date,,,,,,,,,,,,,,
2010-01-01,33,40,1,0,0,0,0.03,3.36,11:55 PM,11:59 PM,0.0,0.0,21.9,w
2010-01-02,17,34,1,0,0,0,0.02,12.53,11:49 PM,11:49 PM,0.2,0.0,34.9,w
2010-01-03,17,22,1,0,0,0,0.00,14.32,12:54 PM,10:55 AM,0.0,0.0,38.9,nw
2010-01-04,19,30,0,0,0,1,0.00,10.74,2:49 AM,5:22 PM,0.0,0.0,23.9,w
2010-01-05,20,30,1,0,0,0,0.00,9.84,10:06 PM,10:47 AM,0.0,0.0,23.9,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-27,62,78,0,0,0,1,0.00,NaN,NaN,NaN,0.0,0.0,NaN,n
2021-09-28,61,71,0,0,1,0,0.27,NaN,NaN,NaN,0.0,0.0,NaN,n
2021-09-29,54,68,0,0,0,1,0.00,NaN,NaN,NaN,0.0,0.0,NaN,n


In [12]:
turnstile['month']

0     Oct-14
1     Nov-14
2     Dec-14
3     Jan-15
4     Feb-15
       ...  
80    Jun-21
81    Jul-21
82    Aug-21
83    Sep-21
84    Oct-21
Name: month, Length: 85, dtype: object

In [14]:
turnstile['month_'] = turnstile['month'].str.slice(0,3).astype(str)
turnstile['year'] = turnstile['month'].str.slice(4,9).astype(str)

In [18]:
turnstile['date'] = pd.to_datetime(turnstile['month_'].astype(str) + turnstile['year'], format='%b%y')

In [20]:
turnstile.drop(['month','month_','year'], axis=1,inplace=True)

In [23]:
cols_list = list(turnstile.columns)

In [25]:
cols_list = sorted(cols_list)

In [26]:
turnstile.columns = cols_list

['103_st_entr',
 '103_st_exits',
 '104_st_entr',
 '104_st_exits',
 '111_st_entr',
 '111_st_exits',
 '116_st_entr',
 '116_st_exits',
 '125_st_entr',
 '125_st_exits',
 '135_st_entr',
 '135_st_exits',
 '145_st_entr',
 '145_st_exits',
 '14_st_entr',
 '14_st_exits',
 '155_st_entr',
 '155_st_exits',
 '163_st_amsterdm_entr',
 '163_st_amsterdm_exits',
 '168_st_entr',
 '168_st_exits',
 '175_st_entr',
 '175_st_exits',
 '181_st_entr',
 '181_st_exits',
 '190_st_entr',
 '190_st_exits',
 '23_st_entr',
 '23_st_exits',
 '34_st_entr',
 '34_st_exits',
 '42_st_entr',
 '42_st_exits',
 '50_st_entr',
 '50_st_exits',
 '59_st_entr',
 '59_st_exits',
 '72_st_entr',
 '72_st_exits',
 '80_st_entr',
 '80_st_exits',
 '81_st_museum_entr',
 '81_st_museum_exits',
 '86_st_entr',
 '86_st_exits',
 '88_st_entr',
 '88_st_exits',
 '96_st_entr',
 '96_st_exits',
 'Total Sum of entries',
 'Total Sum of exits',
 'aqueduct_n_cond_entr',
 'aqueduct_n_cond_exits',
 'aqueduct_racetr_entr',
 'aqueduct_racetr_exits',
 'beach_105_st_en

In [36]:
turnstile

,103_st_entr,103_st_exits,104_st_entr,104_st_exits,111_st_entr,111_st_exits,116_st_entr,116_st_exits,125_st_entr,125_st_exits,...,utica_av_exits,van_siclen_av_entr,van_siclen_av_exits,w_4_st_wash_sq_entr,w_4_st_wash_sq_exits,world_trade_ctr_entr,world_trade_ctr_exits,Total Sum of entries,Total Sum of exits,date
0,1339771034,1175549287,312180061,439393083,119036167,159289252,630340462,715496936,34600927541,28576408139,...,352333513,536869121,289711145,265543250,650228597,417383539,155881610,7.510325e+10,6.626928e+10,2014-10-01
1,1938985908,1699319389,341887665,480915802,1542005,1933245,100165647,104067709,2304342370,1881579345,...,396880757,623155442,337136512,380111665,931137920,587099717,219494659,6.638261e+10,4.884205e+10,2014-11-01
2,2015105131,1763593480,413351610,581215713,1594740,1951644,107104106,110268079,2340107113,1910565943,...,553710871,836184568,451403418,235641584,526378086,607005384,227169486,7.262407e+10,5.524914e+10,2014-12-01
3,2047822007,1789884464,479972302,674538985,1577794,1999311,130839543,136310022,4672301998,3836881371,...,567261067,844790583,454299234,24350890,65934335,637029840,238563612,7.754271e+10,5.936920e+10,2015-01-01
4,1839556877,1605513283,415081380,582665790,3552727,2663839,129556252,135031837,4206643577,3454494123,...,509519078,768828853,412998135,22324713,64198566,564490383,211646611,6.861336e+10,5.228385e+10,2015-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2728777474,2165257658,86834687,53216060,9059741280,3019898880,13343992,12451586,56461749,56947067,...,1173194301,1257504992,648242714,61004393,189249313,176436442,325257672,7.770000e+11,7.650000e+11,2021-06-01
81,2808432153,2229252113,807771,710081,9263623316,3089554922,14818207,14259441,61089886,62455605,...,1225081537,1301960949,671656680,43119749,134905643,183450148,340369820,8.040000e+11,7.900000e+11,2021-07-01
82,2813225284,2233995712,2349746,1984528,9419753742,3144993369,16333822,16442483,64821086,67296817,...,1218495684,1297765490,669929580,3844317,14464713,188579058,352413376,8.030000e+11,7.910000e+11,2021-08-01
83,2712397275,2154883947,3702371,3082717,9160406992,3053453312,16525011,16700510,64401219,67573178,...,1188458028,1258435209,650203186,8130813,31570270,183590669,344311417,7.790000e+11,7.690000e+11,2021-09-01


In [35]:
turn

,103_st_entr,103_st_exits,104_st_entr,104_st_exits,111_st_entr,111_st_exits,116_st_entr,116_st_exits,125_st_entr,125_st_exits,...,spring_st_entr,spring_st_exits,utica_av_entr,utica_av_exits,van_siclen_av_entr,van_siclen_av_exits,w_4_st_wash_sq_entr,w_4_st_wash_sq_exits,world_trade_ctr_entr,world_trade_ctr_exits
0,1339771034,1175549287,312180061,439393083,119036167,159289252,630340462,715496936,34600927541,28576408139,...,338734016,91151565,379610520,352333513,536869121,289711145,265543250,650228597,417383539,155881610
1,1938985908,1699319389,341887665,480915802,1542005,1933245,100165647,104067709,2304342370,1881579345,...,500276640,134904923,427692220,396880757,623155442,337136512,380111665,931137920,587099717,219494659
2,2015105131,1763593480,413351610,581215713,1594740,1951644,107104106,110268079,2340107113,1910565943,...,530404721,143454032,595325444,553710871,836184568,451403418,235641584,526378086,607005384,227169486
3,2047822007,1789884464,479972302,674538985,1577794,1999311,130839543,136310022,4672301998,3836881371,...,559846009,152468393,610177079,567261067,844790583,454299234,24350890,65934335,637029840,238563612
4,1839556877,1605513283,415081380,582665790,3552727,2663839,129556252,135031837,4206643577,3454494123,...,492295275,132958926,549008461,509519078,768828853,412998135,22324713,64198566,564490383,211646611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2728777474,2165257658,86834687,53216060,9059741280,3019898880,13343992,12451586,56461749,56947067,...,1219301529,312525502,1199644962,1173194301,1257504992,648242714,61004393,189249313,176436442,325257672
81,2808432153,2229252113,807771,710081,9263623316,3089554922,14818207,14259441,61089886,62455605,...,1249569515,320617894,1253478596,1225081537,1301960949,671656680,43119749,134905643,183450148,340369820
82,2813225284,2233995712,2349746,1984528,9419753742,3144993369,16333822,16442483,64821086,67296817,...,1253043179,321901544,1242989450,1218495684,1297765490,669929580,3844317,14464713,188579058,352413376
83,2712397275,2154883947,3702371,3082717,9160406992,3053453312,16525011,16700510,64401219,67573178,...,1222568070,314595666,1209023751,1188458028,1258435209,650203186,8130813,31570270,183590669,344311417


In [44]:
with open('data/turnstile.pkl', 'wb') as f:
    pkl.dump(turnstile, f)

In [30]:
cols_list.remove('date')